In [1]:
import PyPDF2 as ppy
from addClass import main as ac
from addStudent import main as ass
from addStudClass import main as asc
import pandas as pd
import textract
import pymysql

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/compat/_optional.py:106: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.0.0' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
pdf = ppy.PdfFileReader("./Transcript for Robi.pdf")
pdf2 = ppy.PdfFileReader("UWUnofficialTranscript.pdf")
text = textract.process("UWUnofficialTranscript.pdf")
otherPDF = text.decode("utf-8")

minsu = ppy.PdfFileReader("minsu.pdf")


In [66]:
data = getClasses(minsu)
studentID = data['studentID']
data = data['data']
print(data)
print(studentID)

   Quarter Department ClassNum                ClassTitle Grade  Year
0   AUTUMN     ACADEM      197         RESEARCH SEMINARS   3.5  2017
1   AUTUMN       ENGL      199         W-WRITING NAT SCI   4.0  2017
2   AUTUMN       MATH      125       CALC ANALYT GEOM II   3.9  2017
3   AUTUMN        SOC      270           SOCIAL PROBLEMS   4.0  2017
4   WINTER     ACADEM      198         RESEARCH SEMINARS   4.0  2018
5   WINTER       CHID      332    W-DISABILITY & SOCIETY   4.0  2018
6   WINTER        LSJ      200       INTR LAW SOC & JUST   4.0  2018
7   WINTER        LSJ      322       HUMAN RIGHTS LAT AM   4.0  2018
8   SPRING       ECON      200           INTRO MICROECON   3.9  2018
9   SPRING       JSIS      499        UNDERGRAD RESEARCH   4.0  2018
10  SPRING        SOC      372        CRIME POL AND JUST   4.0  2018
11  SPRING        SOC      376           DRUGS & SOCIETY   3.9  2018
12  AUTUMN      ACCTG      215       ACCTG & FIN REPRTNG   3.9  2018
13  AUTUMN        B E      200   2

In [9]:
classes=data
#     studentID = data['studentID']
#     if (studentID == userID):     
row = data.loc[0]
classesBatch = []
for i in range(len(data)):
    row = data.loc[i]
    classesBatch.append((row['Department'], row['ClassNum'], row['ClassTitle']))
ac.main(classesBatch)

row = data.loc[0]
studClassBatch = []
for i in range(len(data)):
    row = data.loc[i]
    studClassBatch.append((studentID, row['Quarter'], row['ClassNum'], row['Department'], row['ClassTitle'], row['Year']))
asc.main(studClassBatch)

KeyboardInterrupt: 

In [21]:
commitData(pd.DataFrame(studClassBatch[0]), 1636389)

KeyError: 'Department'

In [7]:
info = mergePages(minsu)
info

{'total': [' **************************************************',
  ' * ANY ALTERATION OR MODIFICATION OF THIS RECORD  *',
  ' * OR ANY COPY THEREOF MAY CONSTITUTE A FELONY    *',
  ' * AND/OR LEAD TO STUDENT DISCIPLINARY SANCTIONS. *',
  ' **************************************************',
  '',
  'HIGH SCHOOL FOREIGN LANGUAGE :SPANSH    LEVEL:3',
  'HIGH SCHOOL MATHEMATICS LEVEL:',
  '',
  'ACT SCORE  :  35',
  '',
  '',
  'EXTENSION/INDEPENDENT STDY/ADVANCE PLACEMENT CREDIT:',
  '',
  ' ADVANCE PLACEMENT:',
  '  HSTRY  100 AP WORLD HISTORY       5.0',
  '   (06/01/17)',
  '  MATH   124 AP MATH CALC AB        5.0',
  '   (06/01/17)',
  '  STAT   311 AP STATISTICS          5.0',
  '   (06/01/17)',
  '',
  ' TOTAL EXTENSION/CORRESPONDENCE/AP CREDIT:  15.0',
  ' TOTAL APPLIED TOWARD NEXT DEGREE:          15.0',
  '----------------------------------------------------',
  '',
  '             AUTUMN 2017              UWACAD  1',
  ' ACADEM 197  RESEARCH SEMINARS        2.0   3.5',
  ' EN

In [65]:

def acquirePrefix(a):
    count = 0
    prefix = ''
    hitInt = False
    while hitInt != True:
        element = a[count]
        try:
            int(element)
            hitInt = True
            break
        except:
            prefix = prefix +" "+ element
            count = count + 1
    return prefix
def acquireNum(a):
    count = 0
    num = ''
    hitInt = False
    while hitInt != True:
        element = a[count]
        try:
            int(element)
            return element

        except:
            count = count + 1
    return num

In [20]:
type(data)

pandas.core.frame.DataFrame

In [4]:
def commitData(data, studentID):
    classes=data
#     studentID = data['studentID']
#     if (studentID == userID):     
    row = data.loc[0]
    classesBatch = []
    for i in range(len(data)):
        row = data.loc[i]
        classesBatch.append((row['Department'], row['ClassNum'], row['ClassTitle']))
    ac.main(classesBatch)
    
    row = data.loc[0]
    studClassBatch = []
    for i in range(len(data)):
        row = data.loc[i]
        studClassBatch.append((studentID, row['Quarter'], row['ClassNum'], row['Department'], row['ClassTitle'], row['Year']))
    asc.main(studClassBatch)

    

In [4]:
def mergePages(pages):
    breaks = []
    total = []
    # Look for the word 'Page' on each page 
    # assump|tions: there is only one occurance
    # no valuable data preceds the occurance of 'Page'
    curQtr = ""
    studentNumber = ""
    for i in range(pages.getNumPages()):
        page = pages.getPage(i)
        parsed = page.extractText().split('\n')
        studentNumber = parsed[8]
        for i,x in enumerate(parsed):
            if "Page" in x:
                breaks = i
            if 'CURRENTLY ENROLLED' in x:
                curQtr = x.split('(')[1].split(')')[0].replace('QUARTER, ', "")        
        useful = parsed[breaks + 1: ]
        total.extend(useful)
    return {'total': total, 'curQtr': curQtr.strip(), 'studentID': studentNumber}

In [63]:
def getClasses(pdf):
    information = mergePages(pdf)
    parsed = information['total']
    curQtr = information['curQtr']
    studentID = information['studentID']
    durations = pd.DataFrame(columns = ['Quarter', 'Year', 'StatusWhile', 'index'])
    count = 0
    for i, x in enumerate(parsed):
        if 'AUTUMN' in x or 'WINTER' in x or 'SPRING' in x and 'D E S T R O Y' not in x:
            row = {'Quarter': x.split()[0], 'Year': x.split()[1], 'StatusWhile' : x.split()[2], 'index': i}
    #         y = x.split()
            durations = durations.append(row, ignore_index=True)
            count = count +1

    classes = pd.DataFrame(columns = ['Quarter', 'Department', 'ClassNum', 'ClassTitle', 'Grade'])
    for i in durations.iloc:
        ind = i['index'] + 1
        cur = parsed[ind]
        while 'QTR' not in cur and 'EARNED' not in cur and 'D E S T R O Y' not in cur and ind < len(parsed):
            if 'GEN ST' in cur:
                cur = cur.replace('GEN ST' ,"GEN")
            curSplit = cur.split()
            desc = ""
            grade =  curSplit[-1]
            for j in range(2, len(curSplit) - 2):
                desc = desc + " " + curSplit[j]
            if curQtr == str(i['Quarter'] + " " +i['Year']):
                desc = ""
                for j in range(2, len(curSplit) - 1):
                    desc = desc + " " + curSplit[j]
                grade = "IP"
            row = {'Quarter': i['Quarter'].strip(), 'Year': i['Year'].strip(), 'Grade': grade.strip(), 'Department': acquirePrefix(curSplit), 'ClassNum': acquireNum(curSplit), 'ClassTitle' : desc.strip() }
            classes = classes.append(row, ignore_index=True)
            ind = ind + 1
            cur = parsed[ind]
            
    return {'data': classes, 'studentID': studentID}

In [12]:
commitData(data, studentID)